In [28]:
from collections import deque

In [29]:
def load_maze(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    maze_lines = []
    start = None
    end = None

    for line in lines:
        line = line.strip()  # white chars in start and end of line
        if line.startswith("start"):
            start_coords = line[len("start"):].strip()  # del start and white symb
            start_x, start_y = map(int, start_coords.split(', '))
            start = (start_x, start_y)
        elif line.startswith("end"):
            end_coords = line[len("end"):].strip()  
            end_x, end_y = map(int, end_coords.split(', '))
            end = (end_x, end_y)
        else:
            maze_lines.append(list(line))

    maze = [line for line in maze_lines if line]  
    return maze, start, end

In [30]:
file_path = '01_71_51_156.txt'  # path to file
maze, start, end = load_maze(file_path)

In [31]:
import tarfile

def list_files_in_tar(tar_path):
    with tarfile.open(tar_path, 'r:gz') as tar:
        #list the names of all files in the archive
        for member in tar.getmembers():
            print(member.name)

tar_path = 'dataset(1).tar.gz'
list_files_in_tar(tar_path)


dataset
dataset/84.txt
dataset/0.txt
dataset/42.txt
dataset/332.txt
dataset/36.txt
dataset/72.txt
dataset/114.txt
dataset/6.txt
dataset/220.txt
dataset/4.txt
dataset/26.txt


In [32]:
import tarfile  

def load_maze_from_tar(tar_path, file_path_inside_tar):

    try:
        tar = tarfile.open(tar_path, 'r:gz')  # open the archive for reading
    except:
        print('Could not open the archive.')
        return None, None, None  
    
    
    member = tar.getmember(file_path_inside_tar)  
    file = tar.extractfile(member)  
    content = file.read().decode('utf-8')  # read and decode the file content
    

    lines = content.splitlines()  # split the content into lines
    maze_lines = []  
    start = None  
    end = None  

    for line in lines:
        line = line.strip()  # remove spaces at start and end of the line
        if line.startswith("start"):
            start_coords = line.split("start")[1].strip()  # get the part after start
            start_x, start_y = map(int, start_coords.split(','))  
            start = (start_x, start_y)  # save start position
        elif line.startswith("end"):
            end_coords = line.split("end")[1].strip()  
            end_x, end_y = map(int, end_coords.split(','))  
            end = (end_x, end_y)  
        else:
            maze_lines.append(list(line))  

    maze = [line for line in maze_lines if line]  
    return maze, start, end  

tar_path = 'dataset(1).tar.gz'
file_path_inside_tar = 'dataset/36.txt'  
maze, start, end = load_maze_from_tar(tar_path, file_path_inside_tar)



In [33]:
def print_maze(maze, path=None, start=None, end=None, visited=None, frontier=None):
    symbols = {'path': 'o', 'visited': '#', 'frontier': '*', 'empty': ' ', 'wall': 'X', 'start': 'S', 'end': 'E'}
    # if not prazdna mnozina
    if path is None:
        path = set()
    if visited is None:
        visited = set()
    if frontier is None:
        frontier = set()

    # symbols
    for y, row in enumerate(maze):
        for x, cell in enumerate(row):
            symbol = symbols['empty'] if cell == ' ' else symbols['wall']
            position = (x, y)
            if position == start:
                symbol = symbols['start']
            elif position == end:
                symbol = symbols['end']
            elif position in path:
                symbol = symbols['path']
            elif position in visited:
                symbol = symbols['visited']
            elif position in frontier:
                symbol = symbols['frontier']
            print(symbol, end='')
        print()
    print('-' * 40)

In [34]:
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent


In [35]:
def reconstruct_path(node):
    path = []
    while node:
        path.append(node.state)
        node = node.parent
    return path[::-1]  # from begining to the end

In [36]:
def bfs(maze, start, end):
    start_node = Node(start)
    if start == end:
        return [start], set()  # if same then 0 path

    frontier = deque([start_node])
    explored = set()
    visited = set([start])

    while frontier:
        current_node = frontier.popleft()
        current_state = current_node.state
        explored.add(current_state)

        if current_state == end:
            return reconstruct_path(current_node), explored  #return path and explored

        for direction in [(0, 1), (1, 0), (0, -1), (-1, 0)]:  # down right up left
            x, y = current_state
            next_state = (x + direction[0], y + direction[1])

            if (0 <= next_state[1] < len(maze) and
                0 <= next_state[0] < len(maze[0]) and
                next_state not in explored and
                maze[next_state[1]][next_state[0]] != 'X'):

                visited.add(next_state)
                child_node = Node(next_state, current_node)
                frontier.append(child_node)

    return None, explored  


path, visited = bfs(maze, start, end)

if path:
    print("Path found:")
    print_maze(maze, path=path, start=start, end=end, visited=visited)
    print(f"Nodes expanded: {len(visited)}")
    print(f"Path length: {len(path)}")
else:
    print("No path found")

Path found:
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X          #######################################X###X
X  XX XXXX  #X##XXXXX#XXXXX##X##XXXX###XXXX#X#X###X#X#X
X X          #########X###X###X#######XooooooSX#X#X#X#X
X   XX X  XXXXXX#XXXX#XoooXX##XXX##X#oooXX#XXX##X#XXX#X
X       X    EooooooooooXoooooooooooooX#########X#####X
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
----------------------------------------
Nodes expanded: 149
Path length: 37


In [37]:
import random

In [38]:
def random_search(maze, start, end, max_attempts=1000):
    attempt = 0
    while attempt < max_attempts:
        current_node = Node(start)
        visited = set([start])

        while current_node.state != end:
            x, y = current_node.state
            directions = [(x + dx, y + dy) for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]
                          if 0 <= x + dx < len(maze[0]) and 0 <= y + dy < len(maze)
                          and (x + dx, y + dy) not in visited and maze[y + dy][x + dx] != 'X']

            if not directions:
                break  #cant go anywhere

            next_state = random.choice(directions)
            visited.add(next_state)
            current_node = Node(next_state, current_node)

            if next_state == end:
                return reconstruct_path(current_node), visited

        attempt += 1 

    return None, visited  # didnt work


path, visited = random_search(maze, start, end)

if path:
    print("Path found by Random Search after {} attempts:".format(len(visited)))
    print_maze(maze, path=path, start=start, end=end, visited=visited)
    print(f"Nodes expanded: {len(visited)}")
    print(f"Path length: {len(path)}")
else:
    print("No path found by Random Search after the maximum number of attempts")

Path found by Random Search after 43 attempts:
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X                    oooooooooooooooo             X   X
X  XX XXXX   X  XXXXXoXXXXXooX  XXXXoo XXXX X X   X X X
X X                  oX   X   X      oXooooooSX X X X X
X   XX X  XXXXXX XXXXoX   XX  XXX  X oooXX XXX  X XXX X
X       X    Eoooooooo  X             X         X     X
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
----------------------------------------
Nodes expanded: 43
Path length: 43


In [39]:
def dfs(maze, start, end):
    start_node = Node(start)
    stack = [start_node]  # stack fr
    visited = set([start])  

    while stack:
        current_node = stack.pop()  # take node from vrchol stacku 
        current_state = current_node.state

        if current_state == end:
            return reconstruct_path(current_node), visited  # path explored

        
        x, y = current_state
        for direction in [(0, 1), (1, 0), (0, -1), (-1, 0)]: 
            next_state = (x + direction[0], y + direction[1])

            if (0 <= next_state[1] < len(maze) and
                0 <= next_state[0] < len(maze[0]) and
                next_state not in visited and
                maze[next_state[1]][next_state[0]] != 'X'):

                visited.add(next_state)
                stack.append(Node(next_state, current_node))  # new node on vrchol of stack

    return None, visited  # didnt work

path, visited = dfs(maze, start, end)
if path:
    print("Path found by DFS:")
    print_maze(maze, path=path, start=start, end=end, visited=visited)
    print(f"Nodes expanded: {len(visited)}")
    print(f"Path length: {len(path)}")
else:
    print("No path found by DFS")


Path found by DFS:
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Xoooooooooooooooooooooooooooooo#                  X   X
Xo#XX#XXXX###X##XXXXX#XXXXX##XooXXXX#  XXXX#X#X   X X X
XoXoooooooooooooooooooX   X   Xoooooo#XooooooSX X X X X
XoooXX#X##XXXXXX#XXXXoX   XX  XXX##XooooXX#XXX  X XXX X
X###    X    Eoooooooo# X           ##X#        X     X
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
----------------------------------------
Nodes expanded: 112
Path length: 83


In [40]:
import heapq

In [41]:
class Node:
    def __init__(self, state, parent=None, cost=0):
        self.state = state
        self.parent = parent
        self.cost = cost  # for heuristic

    def __lt__(self, other):
        return self.cost < other.cost  # by cost


In [42]:
def heuristic(a, b):
    #manhetten lenghth
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

In [43]:

def greedy_search(maze, start, end):
    start_node = Node(start, None, heuristic(start, end))
    frontier = []
    heapq.heappush(frontier, start_node)  # less coustly in start
    visited = set([start])

    while frontier:
        current_node = heapq.heappop(frontier)
        current_state = current_node.state

        if current_state == end:
            return reconstruct_path(current_node), visited

        x, y = current_state
        for direction in [(0, 1), (1, 0), (0, -1), (-1, 0)]: 
            next_state = (x + direction[0], y + direction[1])

            if (0 <= next_state[1] < len(maze) and
                0 <= next_state[0] < len(maze[0]) and
                next_state not in visited and
                maze[next_state[1]][next_state[0]] != 'X'):

                visited.add(next_state)
                next_node = Node(next_state, current_node, heuristic(next_state, end))
                heapq.heappush(frontier, next_node)

    return None, visited
path, visited = greedy_search(maze, start, end)
if path:
    print("Path found by Greedy Search:")
    print_maze(maze, path=path, start=start, end=end, visited=visited)
    print(f"Nodes expanded: {len(visited)}")
    print(f"Path length: {len(path)}")
else:
    print("No path found by Greedy Search")

Path found by Greedy Search:
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X                                                 X   X
X  XX XXXX   X  XXXXX XXXXX  X  XXXX   XXXX#X#X   X X X
X X                   X###X   X      #X# #oooSX X X X X
X   XX X  XXXXXX#XXXX#XoooXX##XXX##X#oooXXoXXX  X XXX X
X       X    EooooooooooXoooooooooooooXoooo#    X     X
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
----------------------------------------
Nodes expanded: 55
Path length: 39


In [44]:
class Node:
    def __init__(self, state, parent=None, g=0, f=0):
        self.state = state
        self.parent = parent
        self.g = g # cost of path from start to current
        self.f = f # from start to end throu current = g+ heuristic

    def __lt__(self, other):
        return self.f < other.f


In [45]:
def a_star(maze, start, end):
    start_node = Node(start, None, 0, heuristic(start, end))
    frontier = []
    heapq.heappush(frontier, (start_node.f, start_node))  # by f value
    visited = set([start])
    g_costs = {start: 0}
    while frontier:
        current_f, current_node = heapq.heappop(frontier) # remuves with smolest cost
        current_state = current_node.state

        if current_state == end:
            return reconstruct_path(current_node), visited

        for direction in [(0, 1), (1, 0), (0, -1), (-1, 0)]:  
            x, y = current_state
            next_state = (x + direction[0], y + direction[1])

            if (0 <= next_state[1] < len(maze) and
                0 <= next_state[0] < len(maze[0]) and
                maze[next_state[1]][next_state[0]] != 'X'):

                new_g = g_costs[current_state] + 1  # cost of muve 1
                if next_state not in g_costs or new_g < g_costs[next_state]:
                    g_costs[next_state] = new_g
                    f = new_g + heuristic(next_state, end)
                    next_node = Node(next_state, current_node, new_g, f)
                    heapq.heappush(frontier, (f, next_node))
                    visited.add(next_state)

    return None, visited

path, visited = a_star(maze, start, end)
if path:
    print("Path found by a_star:")
    print_maze(maze, path=path, start=start, end=end, visited=visited)
    print(f"Nodes expanded: {len(visited)}")
    print(f"Path length: {len(path)}")
else:
    print("No path found by a_star")

Path found by a_star:
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X                                          # #    X   X
X  XX XXXX   X  XXXXX XXXXX  X #XXXX## XXXX#X#X   X X X
X X                   X###X ##X#######XooooooSX X X X X
X   XX X  XXXXXX#XXXX#XoooXX##XXX##X#oooXX#XXX  X XXX X
X       X    EooooooooooXoooooooooooooX######   X     X
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
----------------------------------------
Nodes expanded: 70
Path length: 37
